## **Environment setup.**

In [ ]:
!pip3 install tqdm torch torchvision torchaudio --index-url https://download.pytorch.org/whl/cu118
!pip3 install GPUtil netCDF4 geopandas geocube dask_ml zarr
!pip3 install cuml-cu11 --extra-index-url=https://pypi.nvidia.com

## **Pull code from GitHub.**

In [ ]:
!rm -Rf /content/learnergy/
!rm -Rf /content/SIT_FUSE
!git clone https://github.com/nlahaye/learnergy.git
!git clone https://github.com/nlahaye/SIT_FUSE.git

## **Mount GDrive - TODO change out for VRT access**

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

## **Run Initial Feature Extraction & Top Level Clustering**

In [ ]:
%env PYTHONPATH=$PYTHONPATH:/content/learnergy/:/content/SIT_FUSE/
!cd SIT_FUSE; torchrun --nnodes=1 --nproc_per_node=1 dbn_learnergy.py --yaml /content/SIT_FUSE/config/dbn/palm_oil_dbn_colab.yaml

## **Discretize Model Output - Assign Top Level Labels/Clusters**

In [ ]:
%env PYTHONPATH=$PYTHONPATH:/content/learnergy/:/content/SIT_FUSE/
!cd SIT_FUSE; python3 discretize_clusters.py --yaml /content/SIT_FUSE/config/cluster/discretize_palm_oil_colab.yaml

## **Visualize Initial Results**

In [ ]:
# import image module
from IPython.display import Image

# get the image
Image(url="/content/drive/MyDrive/oil_palm_data_science/ML_Test/ucayali_condensed_stacks_2020_h1v1.tiff.clustoutput_test.data_150clusters.png", width=300, height=300)

## **Create Heirarchical Tree/Dendrogram of Labels - Assign More Precise Labels**

In [ ]:
%env PYTHONPATH=$PYTHONPATH:/content/learnergy/:/content/SIT_FUSE/
!cd SIT_FUSE; torchrun --nnodes=1 --nproc_per_node=1 dbn_learnergy_heirarchical.py --yaml /content/SIT_FUSE/config/dbn/palm_oil_dbn_colab.yaml

## **GeoTiff/Intermediate Product Generation**

In [ ]:
%env PYTHONPATH=$PYTHONPATH:/content/learnergy/:/content/SIT_FUSE/
!cd SIT_FUSE/postprocessing/; python3 generate_cluster_geotiffs.py --yaml ../config/postprocess/palm_oil_geotiff_gen_colab.yaml

## **Generate Final Product**

In [ ]:
%env PYTHONPATH=$PYTHONPATH:/content/learnergy/:/content/SIT_FUSE/
!cd SIT_FUSE/postprocessing/; python3 contour_and_fill.py --yaml ../config/postprocess/palm_oil_contour_colab.yaml